In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 54.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=233364087e20739eeb3c67a93b729e9d28ab21a6171717214761c7977bce5539
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[*]').appName("Transformacao com Spark").getOrCreate()

spark

In [4]:
!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip" && unzip semana-1.zip -d dados/

--2022-12-03 03:02:23--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 54.231.168.89, 52.216.131.147, 52.216.89.220, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|54.231.168.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip’

semana-1.zip        100%[===================>]  18.10M  21.1MB/s    in 0.9s    

2022-12-03 03:02:24 (21.1 MB/s) - ‘semana-1.zip’ saved [18975214/18975214]

Archive:  semana-1.zip
  inflating: dados/dataset_bruto.json  


In [5]:
dados = spark.read.json('/content/dados/dataset_bruto.json')

In [6]:
dados.show()

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
|{0, [20], [15], [...|[{2de09d46-dc0d-4...|{3c7057f5-0923-42...|
|{3, [43], [43], [...|[{147a80d9-cd40-4...|{5a9736b5-aaa0-4a...|
|{2, [42], [42], [...|[{35740004-063d-4...|{ec48d96a-137c-49...|
|{0, [], [12], [0]...|[{6d3d2aec-c96f-4...|{dad7db63-e19c-44...|
|{1, [41], [41], [...|[{3d404069-418e-4...|{a845f35f-3ab3-46...|
|{3, [43], [43], [...|[{5b8a8de7-2088-4...|{8c8de313-45d5-4c...|
|{0, [10], [4], [0...|[{5385da42-8276-4...|{0b1d4ee0-2bc3-4c...|
|{0, [12], [12], [...|[{7

In [7]:
dados.count()

89083

In [8]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [9]:
#Filtrando o Data Frame para eliminar as colunas 'imagens' e 'usuario'

anuncio = dados.select('anuncio.*')

anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [10]:
#Analisando os dados {'tipo_uso':['Comercial', 'Residencial'], 'tipo_anuncio':['Usado', 'Lançamento'], 'tipo_unidade':['Outros', 'Apartamento', 'Casa']}

anuncio.groupBy('tipo_uso').count().show()
anuncio.groupBy('tipo_anuncio').count().show()
anuncio.groupBy('tipo_unidade').count().show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+



In [11]:
#Filtrando o Data Frame para elimenar os dados {'tipo_uso':['Residencial'], 'tipo_anuncio':['Lançamento'], 'tipo_unidade':['Outros', 'Casa']}

anuncio = anuncio.filter("tipo_uso == 'Residencial'")
anuncio = anuncio.filter("tipo_unidade == 'Apartamento'")
anuncio = anuncio.filter("tipo_anuncio == 'Usado'")

anuncio.show()
anuncio.count()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

66562

In [12]:
from pyspark.sql import functions as f

In [13]:
#Analisando os a quantidade de elementos das listas registradas nas colunas 'quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util'

caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

for i in caracteristicas:
  anuncio.select(f.size(f.col(i)).alias(i)).groupBy(i).count().show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



In [14]:
#Convertendo os registros das colunas 'quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util' de lista para elemento
#OBS: No caso de listas com 1 elemento registra-se o elemento; No caso de listas vazias registra-se null

caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

anuncio = anuncio.select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])

anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [15]:
#Verifica-se que nem todos os valores de venda estão na primeira célula
valores = anuncio.select('id', 'valores.condominio', 'valores.iptu', 'valores.tipo', 'valores.valor')
valores.show()
valores.groupBy('tipo').count().show()

#Como os valores de condominio e iptu são iguais para venda e aluguel não recebem tratamento
caracteristicas = ['condominio', 'iptu']
valores = valores.select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in valores.columns])
valores.show()

#Tratamento dos dados de valor de compra
from pyspark.sql.functions import when
valores = valores.withColumn("valor", when(valores.tipo[0]=='Venda', valores.valor[0]).when(valores.tipo[1]=='Venda', valores.valor[1]))
valores = valores.withColumn("tipo", when(valores.tipo[0]=='Venda', valores.tipo[0]).when(valores.tipo[1]=='Venda', valores.tipo[1]))
valores.show()
valores.groupBy('tipo').count().show()

+--------------------+----------+------+-------+-------+
|                  id|condominio|  iptu|   tipo|  valor|
+--------------------+----------+------+-------+-------+
|d2e3a3aa-09b5-45a...|     [245]|[null]|[Venda]|[15000]|
|085bab2c-87ad-452...|       [0]|   [0]|[Venda]|[15000]|
|18d22cbe-1b86-476...|       [0]|   [0]|[Venda]|[20000]|
|bed8a354-9317-442...|     [285]|[null]|[Venda]|[20000]|
|12a13315-d67f-48f...|     [245]|[null]|[Venda]|[15000]|
|a2e6d7a5-0ff0-484...|     [285]|[null]|[Venda]|[20000]|
|a6e3173b-c950-4db...|     [250]|[null]|[Venda]|[15000]|
|2e6e5dfb-206c-496...|     [245]|[null]|[Venda]|[15000]|
|99f8d0f9-95a4-461...|     [245]|[null]|[Venda]|[15000]|
|b3f44c1a-2d50-4d5...|     [240]|[null]|[Venda]|[17999]|
|dc99d9e1-4c63-41f...|       [0]|   [0]|[Venda]|[15000]|
|aa3606d8-8bcd-45e...|     [240]|   [0]|[Venda]|[19999]|
|df80b0d5-677c-4be...|     [245]|[null]|[Venda]|[19999]|
|3d6c0218-9b7d-474...|     [290]|[null]|[Venda]|[25000]|
|304266cc-62d1-4ce...|     [285

In [16]:
anuncio = anuncio.join(valores, 'id', how='left').drop('valores')
anuncio.show()

+--------------------+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|valor|
+--------------------+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+
|09c8ba2d-dbf1-487...|    3|      null|       55|        1|[Academia, Churra...|{Todos os Santos,...|      2|     1|       Usado| Apartamento|Residencial|   1|         0|   0|Venda|26000|
|111472a2-afa1-4a7...|    4|        55|       55|        2|[Academia, Churra...|{Recreio dos Band...|      2|     1|       Usado| Apartamento|Residencial|   1|         0|   0|Venda|30000|
|304266cc-62d1-4ce...|    3|        43|       43|        1|[

In [17]:
endereco = anuncio.select('id', 'endereco.bairro', 'endereco.zona')
endereco.show()

+--------------------+--------------------+----------+
|                  id|              bairro|      zona|
+--------------------+--------------------+----------+
|d2e3a3aa-09b5-45a...|           Paciência|Zona Oeste|
|085bab2c-87ad-452...|           Paciência|Zona Oeste|
|18d22cbe-1b86-476...|           Guaratiba|Zona Oeste|
|bed8a354-9317-442...|              Cosmos|Zona Oeste|
|12a13315-d67f-48f...|           Guaratiba|Zona Oeste|
|a2e6d7a5-0ff0-484...|              Cosmos|Zona Oeste|
|a6e3173b-c950-4db...|           Paciência|Zona Oeste|
|2e6e5dfb-206c-496...|           Guaratiba|Zona Oeste|
|99f8d0f9-95a4-461...|           Paciência|Zona Oeste|
|b3f44c1a-2d50-4d5...|           Guaratiba|Zona Oeste|
|dc99d9e1-4c63-41f...|  Pedra de Guaratiba|Zona Oeste|
|aa3606d8-8bcd-45e...|              Cosmos|Zona Oeste|
|df80b0d5-677c-4be...|        Campo Grande|Zona Oeste|
|3d6c0218-9b7d-474...|          Santa Cruz|Zona Oeste|
|304266cc-62d1-4ce...|           Guaratiba|Zona Oeste|
|111472a2-

In [18]:
anuncio = anuncio.join(endereco, 'id', how='left').drop('endereco')
anuncio.show()
anuncio.count()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+--------------------+----------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|valor|              bairro|      zona|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+--------------------+----------+
|09c8ba2d-dbf1-487...|    3|      null|       55|        1|[Academia, Churra...|      2|     1|       Usado| Apartamento|Residencial|   1|         0|   0|Venda|26000|     Todos os Santos|Zona Norte|
|111472a2-afa1-4a7...|    4|        55|       55|        2|[Academia, Churra...|      2|     1|       Usado| Apartamento|Residencial|   1|         0|   0|Venda|30000|Recreio dos Bande...|Zona Oeste|
|3042

66562

In [19]:
anuncio = anuncio.drop('caracteristicas')

In [20]:
anuncio.write.parquet(
    path='/content/dados/dataset_parquet',
    mode='overwrite',
)

In [21]:
anuncio.write.csv(
    path='/content/dados/dataset.csv',
    mode='overwrite',
    sep=';',
    header=True
)

In [22]:
%%time
anuncio = spark.read.parquet(
    '/content/dados/dataset_parquet',
    sep=';',
    inferSchema=True,
    header=True
)

CPU times: user 4.15 ms, sys: 10 µs, total: 4.16 ms
Wall time: 261 ms


In [23]:
%%time
anuncio = spark.read.csv(
    '/content/dados/dataset.csv',
    sep=';',
    inferSchema=True,
    header=True
)

CPU times: user 18.5 ms, sys: 1.23 ms, total: 19.7 ms
Wall time: 1.72 s
